# Pennylane

In [ ]:
# Comentar en caso de hacer una instalación local

!pip install pennylane

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

$\bullet$ Pennylane es una plataforma de Python para programar computadoras cuánticas. Funciona con simuladores o con hardware cuántico.

$\bullet$ Los circuitos se representan mediante *nodos cuánticos*  (*quantum nodes*).

$\bullet$ Los circuitos son funciones.

$\bullet$ Cada cable es un qubit.

$\bullet$ Es necesario seleccionar un *device*, que nos dará acceso a un simulador o al hardware corresppndiente.

In [ ]:
# la forma típica e invocar a pennylane es como qml

import pennylane as qml
import numpy as np

## Un circuito genérico

In [ ]:
# Podemos crear un device de esta manera, donde "wires" es el número de qubits,
# "shots", el número de veces a repetir el experimento y el primer argumento es
# el dispositivo a usar

#dev = qml.device('default.qubit', wires=2, shots=1024)


# O bien, podemos crear un circuuto etiquetando a los qubits:

dev = qml.device('default.qubit', wires=['q0', 'q1'], shots=1024)

In [ ]:
# Creamos un circuito:

@qml.qnode(dev)
def qc(phase):
    qml.Hadamard(wires='q0')
    qml.RX(phase[0], wires='q0')
    qml.RY(phase[1], wires='q1')
    qml.CNOT(wires=['q0', 'q1'])
    return qml.expval((qml.PauliZ('q0')+1)/2), qml.var((qml.PauliX('q1')+1)/2),qml.sample((qml.PauliZ('q0')+1)/2),qml.sample((qml.PauliZ('q1')+1)/2), qml.counts((qml.PauliZ('q0')+1)/2), qml.counts((qml.PauliZ('q1')+1)/2)

phase = [0,0]

In [ ]:
drawer = qml.draw(qc)
print(drawer(phase))

In [ ]:
fig, ax = qml.draw_mpl(qc,style='',decimals=2,wire_order=[1,0])(phase)
fig.show()

In [ ]:
resultados = qc(phase)

In [ ]:
resultados[-1]

## Lo que podemos obtener

### El estado final del sistema:

*Note que shots queda fuera de la declaración del dispositivo.*

In [ ]:
dev = qml.device('default.qubit', wires=2)

# Creamos un circuito:

@qml.qnode(dev)
def qc():
    qml.Hadamard(0)
    qml.CNOT(wires=[0, 1])
    return qml.state()


In [ ]:
drawer = qml.draw(qc)
print(drawer())

In [ ]:
resultados = qc()

Como esperábamos, el estado final es $$\tfrac{1}{\sqrt{2}}\big(|00\rangle+|11\rangle\big)$$

In [ ]:
resultados

### Las probabilidades de los estados en la base computacional

In [ ]:
dev = qml.device('default.qubit', wires=2, shots=480)

# Creamos un circuito:

@qml.qnode(dev)
def qc():
    qml.Hadamard(0)
    qml.CNOT(wires=[0, 1])
    return qml.probs()

resultados = qc()
resultados

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize = (10,4))

plt.bar(range(len(resultados)), resultados)

idx = np.asarray([i for i in range(len(resultados))])

ax.set_xticks(idx)

ax.set_xticklabels(idx, rotation=65)

ax.xaxis.set_major_formatter(StrMethodFormatter("{x:02b}"))

# Add a title and labels
plt.ylabel('probability')


# Display the plot
plt.show()

### Las cuentas de las observables solicitadas


In [ ]:
dev = qml.device('default.qubit', wires=2, shots=2048)

# Creamos un circuito:

@qml.qnode(dev)
def qc():
    qml.Hadamard(0)
    qml.CNOT(wires=[0, 1])
    return qml.counts()

resultados = qc()
resultados

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize = (10,4))

# Convert the dictionary values to a list
heights = list(resultados.values())

plt.bar(range(len(resultados)), heights)

idx = np.asarray([i for i in range(len(resultados))])

ax.set_xticks(idx)

ax.set_xticklabels(idx, rotation=65)

ax.xaxis.set_major_formatter(StrMethodFormatter("{x:02b}"))

# Add a title and labels
plt.ylabel('counts')


# Display the plot
plt.show()

### El promedio y la varianza de una observable

En general, el valor esperado (o valor promedio) de una observable/operador $\hat A$ es
$$\langle A\rangle =\langle\psi|\hat A|\psi\rangle.$$

\\

La varianza es

$$\sigma_A =\langle \hat A^2\rangle-\langle\hat A\rangle^2.$$

\\

En el siguiente circuito calculamos el valor esperado de $\hat Z$, con $|\psi\rangle=\frac{1}{\sqrt{2}}\big(|0\rangle+|1\rangle\big).$

In [ ]:
dev = qml.device('default.qubit', wires=1, shots=4096)

# Creamos un circuito:

@qml.qnode(dev)
def qc():
    qml.Hadamard(0)
    return qml.expval(qml.PauliZ(0)), qml.var(qml.PauliZ(0))

resultados = qc()
resultados

In [ ]:
drawer = qml.draw(qc)
print(drawer())

In [ ]:
fig, ax = qml.draw_mpl(qc,style='sketch')()
fig.show()

## Algunas compuertas incluidas

In [ ]:
# Creamos un circuito:

@qml.qnode(dev)
def qc(phase):
    qml.Hadamard(0)
    qml.RX(phase[0], wires=0)
    qml.RY(phase[1], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.PhaseShift(phase[2], wires=0)
    qml.PauliX(wires=0)
    qml.PauliY(wires=0)
    qml.PauliZ(wires=0)
    qml.T(wires=0)
    qml.S(wires=0)
    qml.adjoint(qml.T(wires=1))
    qml.adjoint(qml.S(wires=1))
    qml.SWAP(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))


phase = [np.pi/3,np.pi/7,np.pi/8]

In [ ]:
#drawer = qml.draw(qc)
#print(drawer(phase))

In [ ]:
fig, ax = qml.draw_mpl(qc,style='',decimals=2,wire_order=[1,0])(phase)
fig.show()

## Cómo crear una compuerta

In [ ]:
# Primero creamos una matriz unitaria

U = [[0, 1], [1, 0]]
matrix = np.matrix(U)

In [ ]:
@qml.qnode(dev)
def qc_test():
    qml.QubitUnitary(U, wires=0)  # incluimos la nueva compuerta, actuando sobre el qubit 1
    qml.Hadamard(0)
    return qml.counts()

In [ ]:
fig, ax = qml.draw_mpl(qc_test,style='',decimals=2,wire_order=[1,0])()
fig.show()

In [ ]:
resultados = qc_test()
resultados